# Final Project Exploration

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import warnings
import data_read
import load_assets

warnings.filterwarnings("ignore")

### Reports load

In [2]:
#Data Series from WRDS
rcfd_data_1 = load_assets.load_wrds_reports('ddss0fpozaxonboe') #series 1 of rcfd
rcfd_data_2 = load_assets.load_wrds_reports('dycfrwcdm9puanhs') #series 2 of rcfd
rcon_data_1 = load_assets.load_wrds_reports('m3pzkcjsgvk26dwa') #series 1 of rcon
rcon_data_2 = load_assets.load_wrds_reports('hwv0m9qml6efztsi') #series 2 of rcon
rcfn_data = load_assets.load_wrds_reports('cipzs5x6g2axzlhe') #rcfn

# Assets

In [7]:
analysis_date = '03/31/2022'
filtered_asset_level_0 = load_assets.clean_assets(rcfd_data_2,'RCFD2170',analysis_date)
filtered_asset_level = load_assets.clean_assets(rcon_data_2,'RCON2170',analysis_date)
filtered_asset_level.head()


,Bank_ID,bank_name,report_date,gross_asset
0,37,BANK OF HANCOCK COUNTY,03/31/2022,87842.0
5,242,FIRST COMMUNITY BANK XENIA-FLORA,03/31/2022,53236.0
10,279,"MINEOLA COMMUNITY BANK, SSB",03/31/2022,368875.0
15,354,BISON STATE BANK,03/31/2022,25077.0
20,457,LOWRY STATE BANK,03/31/2022,65336.0


In [9]:
df_asset = pd.concat([filtered_asset_level_0, filtered_asset_level])
df_asset.head()

,Bank_ID,bank_name,report_date,gross_asset
230,12311,"HUNTINGTON NATIONAL BANK, THE",03/31/2022,176020595.0
537,30810,DISCOVER BANK,03/31/2022,105657408.0
635,35301,STATE STREET BANK AND TRUST COMPANY,03/31/2022,318494000.0
1095,60143,COMERICA BANK,03/31/2022,89142000.0
1170,63069,CITY NATIONAL BANK,03/31/2022,91770285.0


## Creating a dataframe for total/gross asset for each bank

In [9]:
gross_asset = df_asset[['RSSD9001','RSSD9017', 'RSSD9999', 'Total Asset']]

gross_asset  = gross_asset.rename(columns={
    'RSSD9001': 'Bank_ID',
    'RSSD9017': 'bank_name',
    'RSSD9999': 'report_date',
    'Total Asset': 'gross_asset',
})
gross_asset

,Bank_ID,bank_name,report_date,gross_asset
230,12311,"HUNTINGTON NATIONAL BANK, THE",03/31/2022,176020595.0
537,30810,DISCOVER BANK,03/31/2022,105657408.0
635,35301,STATE STREET BANK AND TRUST COMPANY,03/31/2022,318494000.0
1095,60143,COMERICA BANK,03/31/2022,89142000.0
1170,63069,CITY NATIONAL BANK,03/31/2022,91770285.0
...,...,...,...,...
23870,5620869,GENESIS BANK,03/31/2022,95052.0
23875,5650923,WATERFALL BANK,03/31/2022,87152.0
23880,5660146,ENTREBANK,03/31/2022,35261.0
23885,5673191,LOCALITY BANK,03/31/2022,43775.0


# LOANS

## Residential Loans with repricing maturity (secured by first liens)

In [10]:
df_loans_first_lien_domestic = rcon_data_1[['RSSD9001','RSSD9017', 'RSSD9999', 'RCONA564', 'RCONA565', 'RCONA566', 'RCONA567', 'RCONA568', 'RCONA569']]
df_loans_first_lien_domestic = df_loans_first_lien_domestic.rename(columns={
    'RSSD9001': 'Bank_ID',
    'RSSD9017': 'bank_name',
    'RSSD9999': 'report_date',
    'RCONA564': '<3m',
    'RCONA565': '3m-1y',
    'RCONA566': '1y-3y',
    'RCONA567': '3y-5y',
    'RCONA568': '5y-15y',
    'RCONA569': '>15y'
})
df_loans_first_lien_domestic = df_loans_first_lien_domestic[df_loans_first_lien_domestic['report_date'] == '03/31/2022']
df_loans_first_lien_domestic

,Bank_ID,bank_name,report_date,<3m,3m-1y,1y-3y,3y-5y,5y-15y,>15y
0,37,BANK OF HANCOCK COUNTY,03/31/2022,355,354,2430,2229,2641,0
5,242,FIRST COMMUNITY BANK XENIA-FLORA,03/31/2022,683,588,2566,5361,2137,0
10,279,"MINEOLA COMMUNITY BANK, SSB",03/31/2022,74,22,2136,4966,30924,113757
15,354,BISON STATE BANK,03/31/2022,41,230,995,4528,441,5310
20,457,LOWRY STATE BANK,03/31/2022,212,999,2964,11051,2962,245
...,...,...,...,...,...,...,...,...,...
23870,5620869,GENESIS BANK,03/31/2022,0,0,0,0,0,0
23875,5650923,WATERFALL BANK,03/31/2022,0,0,0,0,0,0
23880,5660146,ENTREBANK,03/31/2022,0,0,0,0,0,0
23885,5673191,LOCALITY BANK,03/31/2022,0,0,0,0,0,0


## Loans and Leases (not-secured by first liens)

In [11]:
df_loans_exc_first_lien = rcfd_data_1[['RSSD9001','RSSD9017', 'RSSD9999', 'RCFDA570', 'RCFDA571', 'RCFDA572', 'RCFDA573', 'RCFDA574', 'RCFDA575']]
df_loans_exc_first_lien = df_loans_exc_first_lien.rename(columns={
    'RSSD9001': 'Bank_ID',
    'RSSD9017': 'bank_name',
    'RSSD9999': 'report_date',
    'RCFDA570': '<3m',
    'RCFDA571': '3m-1y',
    'RCFDA572': '1y-3y',
    'RCFDA573': '3y-5y',
    'RCFDA574': '5y-15y',
    'RCFDA575': '>15y'
})
df_loans_exc_first_lien = df_loans_exc_first_lien.dropna()
df_loans_exc_first_lien = df_loans_exc_first_lien[df_loans_exc_first_lien['report_date'] == '03/31/2022']
df_loans_exc_first_lien

,Bank_ID,bank_name,report_date,<3m,3m-1y,1y-3y,3y-5y,5y-15y,>15y
230,12311,"HUNTINGTON NATIONAL BANK, THE",03/31/2022,51195560.0,2317689.0,8818410.0,13942860.0,13029737.0,2818336.0
537,30810,DISCOVER BANK,03/31/2022,61418068.0,9200928.0,10704253.0,3325225.0,3837412.0,3966581.0
635,35301,STATE STREET BANK AND TRUST COMPANY,03/31/2022,30321000.0,724000.0,441000.0,647000.0,3160000.0,0.0
1095,60143,COMERICA BANK,03/31/2022,40106000.0,2164000.0,2258000.0,1708000.0,1221000.0,18000.0
1170,63069,CITY NATIONAL BANK,03/31/2022,19982071.0,4209651.0,5020766.0,3965624.0,4488435.0,1508827.0
...,...,...,...,...,...,...,...,...,...
22697,3382547,DEPARTMENT STORES NATIONAL BANK,03/31/2022,39452.0,0.0,0.0,0.0,0.0,0.0
22787,3402913,BANESCO USA,03/31/2022,235152.0,51990.0,376473.0,414888.0,249055.0,3060.0
23496,3783313,CROSS RIVER BANK,03/31/2022,795756.0,883304.0,1607343.0,3897324.0,61057.0,0.0
23615,4114567,FIRST REPUBLIC BANK,03/31/2022,17758981.0,6042497.0,5830371.0,6531830.0,19691235.0,4020366.0


In [12]:
df_loans_exc_first_lien_domestic = rcon_data_2[['RSSD9001', 'RSSD9017', 'RSSD9999', 'RCONA570', 'RCONA571', 'RCONA572', 'RCONA573', 'RCONA574', 'RCONA575']]
df_loans_exc_first_lien_domestic = df_loans_exc_first_lien_domestic.rename(columns={
    'RSSD9001': 'Bank_ID',
    'RSSD9017': 'bank_name',
    'RSSD9999': 'report_date',
    'RCONA570': '<3m',
    'RCONA571': '3m-1y',
    'RCONA572': '1y-3y',
    'RCONA573': '3y-5y',
    'RCONA574': '5y-15y',
    'RCONA575': '>15y'
})
df_loans_exc_first_lien_domestic = df_loans_exc_first_lien_domestic.dropna()
df_loans_exc_first_lien_domestic = df_loans_exc_first_lien_domestic[df_loans_exc_first_lien_domestic['report_date'] == '03/31/2022']
df_loans_exc_first_lien_domestic

,Bank_ID,bank_name,report_date,<3m,3m-1y,1y-3y,3y-5y,5y-15y,>15y
0,37,BANK OF HANCOCK COUNTY,03/31/2022,1082.0,3574.0,2928.0,3088.0,2106.0,0.0
5,242,FIRST COMMUNITY BANK XENIA-FLORA,03/31/2022,6436.0,2335.0,1715.0,2749.0,767.0,481.0
10,279,"MINEOLA COMMUNITY BANK, SSB",03/31/2022,8861.0,11647.0,16963.0,11508.0,18670.0,5189.0
15,354,BISON STATE BANK,03/31/2022,1023.0,802.0,1041.0,1249.0,489.0,0.0
20,457,LOWRY STATE BANK,03/31/2022,5623.0,5694.0,6740.0,15109.0,6130.0,1116.0
...,...,...,...,...,...,...,...,...,...
23870,5620869,GENESIS BANK,03/31/2022,0.0,4399.0,4441.0,18669.0,32327.0,0.0
23875,5650923,WATERFALL BANK,03/31/2022,3146.0,0.0,128.0,6687.0,14486.0,0.0
23880,5660146,ENTREBANK,03/31/2022,0.0,0.0,0.0,3113.0,0.0,0.0
23885,5673191,LOCALITY BANK,03/31/2022,1043.0,0.0,0.0,5559.0,1710.0,0.0


In [13]:
df_other_loan = pd.concat([df_loans_exc_first_lien_domestic, df_loans_exc_first_lien])
df_other_loan = df_other_loan.sort_index()

# RMBs

In [14]:
df_RMBS = rcfd_data_1[['RSSD9001', 'RSSD9017', 'RSSD9999', 'RCFDA555', 'RCFDA556', 'RCFDA557', 'RCFDA558', 'RCFDA559', 'RCFDA560']]
df_RMBS = df_RMBS.rename(columns={
    'RSSD9001': 'Bank_ID',
    'RSSD9017': 'bank_name',
    'RSSD9999': 'report_date',
    'RCFDA555': '<3m',
    'RCFDA556': '3m-1y',
    'RCFDA557': '1y-3y',
    'RCFDA558': '3y-5y',
    'RCFDA559': '5y-15y',
    'RCFDA560': '>15y'
})
df_RMBS = df_RMBS.dropna()
df_RMBS = df_RMBS[df_RMBS['report_date'] == '03/31/2022']
df_RMBS

,Bank_ID,bank_name,report_date,<3m,3m-1y,1y-3y,3y-5y,5y-15y,>15y
230,12311,"HUNTINGTON NATIONAL BANK, THE",03/31/2022,20700.0,0.0,108066.0,63.0,919381.0,22862348.0
537,30810,DISCOVER BANK,03/31/2022,19.0,815.0,12235.0,59345.0,88858.0,203538.0
635,35301,STATE STREET BANK AND TRUST COMPANY,03/31/2022,5000.0,54000.0,3000.0,0.0,3122000.0,19252000.0
1095,60143,COMERICA BANK,03/31/2022,6000.0,0.0,50000.0,183000.0,172000.0,8967000.0
1170,63069,CITY NATIONAL BANK,03/31/2022,0.0,0.0,11288.0,0.0,271642.0,53862.0
...,...,...,...,...,...,...,...,...,...
22697,3382547,DEPARTMENT STORES NATIONAL BANK,03/31/2022,0.0,0.0,0.0,0.0,0.0,0.0
22787,3402913,BANESCO USA,03/31/2022,0.0,0.0,0.0,0.0,0.0,219612.0
23496,3783313,CROSS RIVER BANK,03/31/2022,0.0,0.0,0.0,0.0,64.0,0.0
23615,4114567,FIRST REPUBLIC BANK,03/31/2022,11521.0,0.0,0.0,3548.0,137112.0,2402824.0


In [15]:
#Form 051 - Domestic
df_RMBS_dom = rcon_data_1[['RSSD9001','RSSD9017', 'RSSD9999', 'RCONA555', 'RCONA556', 'RCONA557', 'RCONA558', 'RCONA559', 'RCONA560']]
df_RMBS_dom = df_RMBS_dom.rename(columns={
    'RSSD9001': 'Bank_ID',
    'RSSD9017': 'bank_name',
    'RSSD9999': 'report_date',
    'RCONA555': '<3m',
    'RCONA556': '3m-1y',
    'RCONA557': '1y-3y',
    'RCONA558': '3y-5y',
    'RCONA559': '5y-15y',
    'RCONA560': '>15y'
})
df_RMBS_dom = df_RMBS_dom.dropna()
df_RMBS_dom = df_RMBS_dom[df_RMBS_dom['report_date'] == '03/31/2022']
df_RMBS_dom

,Bank_ID,bank_name,report_date,<3m,3m-1y,1y-3y,3y-5y,5y-15y,>15y
0,37,BANK OF HANCOCK COUNTY,03/31/2022,0.0,0.0,0.0,0.0,856.0,6021.0
5,242,FIRST COMMUNITY BANK XENIA-FLORA,03/31/2022,0.0,0.0,0.0,0.0,0.0,0.0
10,279,"MINEOLA COMMUNITY BANK, SSB",03/31/2022,2814.0,86.0,0.0,1049.0,26085.0,16737.0
15,354,BISON STATE BANK,03/31/2022,0.0,0.0,0.0,0.0,58.0,0.0
20,457,LOWRY STATE BANK,03/31/2022,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
23870,5620869,GENESIS BANK,03/31/2022,0.0,0.0,0.0,0.0,0.0,0.0
23875,5650923,WATERFALL BANK,03/31/2022,0.0,0.0,0.0,0.0,0.0,6628.0
23880,5660146,ENTREBANK,03/31/2022,0.0,0.0,0.0,0.0,0.0,0.0
23885,5673191,LOCALITY BANK,03/31/2022,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
df_RMBS_Final = pd.concat([df_RMBS_dom, df_RMBS])
df_RMBS_Final = df_RMBS_Final.sort_index()
df_RMBS_Final

,Bank_ID,bank_name,report_date,<3m,3m-1y,1y-3y,3y-5y,5y-15y,>15y
0,37,BANK OF HANCOCK COUNTY,03/31/2022,0.0,0.0,0.0,0.0,856.0,6021.0
5,242,FIRST COMMUNITY BANK XENIA-FLORA,03/31/2022,0.0,0.0,0.0,0.0,0.0,0.0
10,279,"MINEOLA COMMUNITY BANK, SSB",03/31/2022,2814.0,86.0,0.0,1049.0,26085.0,16737.0
15,354,BISON STATE BANK,03/31/2022,0.0,0.0,0.0,0.0,58.0,0.0
20,457,LOWRY STATE BANK,03/31/2022,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
23870,5620869,GENESIS BANK,03/31/2022,0.0,0.0,0.0,0.0,0.0,0.0
23875,5650923,WATERFALL BANK,03/31/2022,0.0,0.0,0.0,0.0,0.0,6628.0
23880,5660146,ENTREBANK,03/31/2022,0.0,0.0,0.0,0.0,0.0,0.0
23885,5673191,LOCALITY BANK,03/31/2022,0.0,0.0,0.0,0.0,0.0,0.0


# Treausurys and Other (non-RMBs)

In [17]:
df_non_RMBS = rcfd_data_2[['RSSD9001','RSSD9017', 'RSSD9999', 'RCFDA549', 'RCFDA550', 'RCFDA551', 'RCFDA552', 'RCFDA553', 'RCFDA554']]
df_non_RMBS = df_non_RMBS.rename(columns={
    'RSSD9001': 'Bank_ID',
    'RSSD9017': 'bank_name',
    'RSSD9999': 'report_date',
    'RCFDA549': '<3m',
    'RCFDA550': '3m-1y',
    'RCFDA551': '1y-3y',
    'RCFDA552': '3y-5y',
    'RCFDA553': '5y-15y',
    'RCFDA554': '>15y'
})
df_non_RMBS = df_non_RMBS.dropna()
df_non_RMBS = df_non_RMBS[df_non_RMBS['report_date'] == '03/31/2022']
df_non_RMBS

,Bank_ID,bank_name,report_date,<3m,3m-1y,1y-3y,3y-5y,5y-15y,>15y
230,12311,"HUNTINGTON NATIONAL BANK, THE",03/31/2022,1351236.0,205700.0,413405.0,878825.0,3382386.0,392127.0
537,30810,DISCOVER BANK,03/31/2022,375727.0,1254282.0,3850470.0,123598.0,7938.0,0.0
635,35301,STATE STREET BANK AND TRUST COMPANY,03/31/2022,12830000.0,6242000.0,20908000.0,17035000.0,5160000.0,343000.0
1095,60143,COMERICA BANK,03/31/2022,0.0,804000.0,1388000.0,638000.0,2005000.0,0.0
1170,63069,CITY NATIONAL BANK,03/31/2022,1725337.0,61661.0,1845305.0,764252.0,1084041.0,217430.0
...,...,...,...,...,...,...,...,...,...
22697,3382547,DEPARTMENT STORES NATIONAL BANK,03/31/2022,0.0,0.0,0.0,0.0,0.0,0.0
22787,3402913,BANESCO USA,03/31/2022,59394.0,0.0,12857.0,30254.0,151294.0,9246.0
23496,3783313,CROSS RIVER BANK,03/31/2022,370.0,1577.0,3386.0,4155.0,80139.0,8000.0
23615,4114567,FIRST REPUBLIC BANK,03/31/2022,1284346.0,169193.0,464500.0,249865.0,723664.0,17839822.0


In [18]:
#Form 051 - Domestic
df_non_RMBS_dom = rcon_data_2[['RSSD9001','RSSD9017','RSSD9999', 'RCONA549', 'RCONA550', 'RCONA551', 'RCONA552', 'RCONA553', 'RCONA554']]
df_non_RMBS_dom = df_non_RMBS_dom.rename(columns={
    'RSSD9001': 'Bank_ID',
    'RSSD9017': 'bank_name',
    'RSSD9999': 'report_date',
    'RCONA549': '<3m',
    'RCONA550': '3m-1y',
    'RCONA551': '1y-3y',
    'RCONA552': '3y-5y',
    'RCONA553': '5y-15y',
    'RCONA554': '>15y'
})
df_non_RMBS_dom = df_non_RMBS_dom.dropna()
df_non_RMBS_dom = df_non_RMBS_dom[df_non_RMBS_dom['report_date'] == '03/31/2022']
df_non_RMBS_dom

,Bank_ID,bank_name,report_date,<3m,3m-1y,1y-3y,3y-5y,5y-15y,>15y
0,37,BANK OF HANCOCK COUNTY,03/31/2022,0.0,0.0,330.0,708.0,26220.0,19212.0
5,242,FIRST COMMUNITY BANK XENIA-FLORA,03/31/2022,600.0,784.0,2268.0,2813.0,11784.0,135.0
10,279,"MINEOLA COMMUNITY BANK, SSB",03/31/2022,0.0,0.0,8190.0,7815.0,16591.0,5854.0
15,354,BISON STATE BANK,03/31/2022,150.0,348.0,0.0,189.0,0.0,0.0
20,457,LOWRY STATE BANK,03/31/2022,0.0,0.0,952.0,479.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
23870,5620869,GENESIS BANK,03/31/2022,0.0,0.0,0.0,0.0,0.0,0.0
23875,5650923,WATERFALL BANK,03/31/2022,0.0,0.0,0.0,0.0,2713.0,0.0
23880,5660146,ENTREBANK,03/31/2022,0.0,0.0,0.0,0.0,0.0,0.0
23885,5673191,LOCALITY BANK,03/31/2022,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
df_treasury_and_others = pd.concat([df_non_RMBS_dom, df_non_RMBS])
df_treasury_and_others = df_treasury_and_others.sort_index()
df_treasury_and_others 

,Bank_ID,bank_name,report_date,<3m,3m-1y,1y-3y,3y-5y,5y-15y,>15y
0,37,BANK OF HANCOCK COUNTY,03/31/2022,0.0,0.0,330.0,708.0,26220.0,19212.0
5,242,FIRST COMMUNITY BANK XENIA-FLORA,03/31/2022,600.0,784.0,2268.0,2813.0,11784.0,135.0
10,279,"MINEOLA COMMUNITY BANK, SSB",03/31/2022,0.0,0.0,8190.0,7815.0,16591.0,5854.0
15,354,BISON STATE BANK,03/31/2022,150.0,348.0,0.0,189.0,0.0,0.0
20,457,LOWRY STATE BANK,03/31/2022,0.0,0.0,952.0,479.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
23870,5620869,GENESIS BANK,03/31/2022,0.0,0.0,0.0,0.0,0.0,0.0
23875,5650923,WATERFALL BANK,03/31/2022,0.0,0.0,0.0,0.0,2713.0,0.0
23880,5660146,ENTREBANK,03/31/2022,0.0,0.0,0.0,0.0,0.0,0.0
23885,5673191,LOCALITY BANK,03/31/2022,0.0,0.0,0.0,0.0,0.0,0.0


# Additional Data (Ignore, I didnt use these for the table)

In [20]:
df_other_MBS = rcfd_data_1[['RSSD9001','RSSD9017', 'RSSD9999', 'RCFDA561', 'RCFDA562']]
df_other_MBS = df_other_MBS .rename(columns={
    'RSSD9001': 'Bank_ID',
    'RSSD9017': 'bank_name',
    'RSSD9999': 'report_date',
    'RCFDA561': '<3y',
    'RCFDA562': '>3y',
})
df_other_MBS = df_other_MBS.dropna()
df_other_MBS  = df_other_MBS[df_other_MBS['report_date'] == '03/31/2022']
df_other_MBS 

,Bank_ID,bank_name,report_date,<3y,>3y
230,12311,"HUNTINGTON NATIONAL BANK, THE",03/31/2022,905975.0,10885639.0
537,30810,DISCOVER BANK,03/31/2022,0.0,0.0
635,35301,STATE STREET BANK AND TRUST COMPANY,03/31/2022,10596000.0,23772000.0
1095,60143,COMERICA BANK,03/31/2022,1727000.0,2868000.0
1170,63069,CITY NATIONAL BANK,03/31/2022,1511646.0,19059340.0
...,...,...,...,...,...
22697,3382547,DEPARTMENT STORES NATIONAL BANK,03/31/2022,0.0,0.0
22787,3402913,BANESCO USA,03/31/2022,635.0,0.0
23496,3783313,CROSS RIVER BANK,03/31/2022,56002.0,160.0
23615,4114567,FIRST REPUBLIC BANK,03/31/2022,1850782.0,5149277.0


In [21]:
df_other_MBS_domestic = rcon_data_2[['RSSD9001','RSSD9017', 'RSSD9999', 'RCONA561', 'RCONA562']]
df_other_MBS_domestic = df_other_MBS_domestic.rename(columns={
    'RSSD9001': 'Bank_ID',
    'RSSD9017': 'bank_name',
    'RSSD9999': 'report_date',
    'RCONA561': '<3y',
    'RCONA562': '>3y',
})
df_other_MBS_domestic  = df_other_MBS_domestic.dropna()
df_other_MBS_domestic  = df_other_MBS_domestic[df_other_MBS_domestic['report_date'] == '03/31/2022']
df_other_MBS_domestic 

,Bank_ID,bank_name,report_date,<3y,>3y
0,37,BANK OF HANCOCK COUNTY,03/31/2022,0.0,0.0
5,242,FIRST COMMUNITY BANK XENIA-FLORA,03/31/2022,0.0,0.0
10,279,"MINEOLA COMMUNITY BANK, SSB",03/31/2022,0.0,10287.0
15,354,BISON STATE BANK,03/31/2022,0.0,0.0
20,457,LOWRY STATE BANK,03/31/2022,0.0,0.0
...,...,...,...,...,...
23870,5620869,GENESIS BANK,03/31/2022,0.0,0.0
23875,5650923,WATERFALL BANK,03/31/2022,0.0,0.0
23880,5660146,ENTREBANK,03/31/2022,0.0,0.0
23885,5673191,LOCALITY BANK,03/31/2022,0.0,0.0


In [22]:
df_other_MBS_total = pd.concat([df_other_MBS, df_other_MBS_domestic])
df_other_MBS_total

,Bank_ID,bank_name,report_date,<3y,>3y
230,12311,"HUNTINGTON NATIONAL BANK, THE",03/31/2022,905975.0,10885639.0
537,30810,DISCOVER BANK,03/31/2022,0.0,0.0
635,35301,STATE STREET BANK AND TRUST COMPANY,03/31/2022,10596000.0,23772000.0
1095,60143,COMERICA BANK,03/31/2022,1727000.0,2868000.0
1170,63069,CITY NATIONAL BANK,03/31/2022,1511646.0,19059340.0
...,...,...,...,...,...
23870,5620869,GENESIS BANK,03/31/2022,0.0,0.0
23875,5650923,WATERFALL BANK,03/31/2022,0.0,0.0
23880,5660146,ENTREBANK,03/31/2022,0.0,0.0
23885,5673191,LOCALITY BANK,03/31/2022,0.0,0.0


In [23]:
df_other_loan_lease = rcfd_data_1[['RSSD9001','RSSD9017', 'RSSD9999', 'RCFDA247']]
df_other_loan_lease = df_other_loan_lease.rename(columns={
    'RSSD9001': 'Bank_ID',
    'RSSD9017': 'bank_name',
    'RSSD9999': 'report_date',
    'RCFDA247': '<1y'

})
df_other_loan_lease = df_other_loan_lease.dropna()
df_other_loan_lease = df_other_loan_lease[df_other_loan_lease['report_date'] == '03/31/2022']
df_other_loan_lease

,Bank_ID,bank_name,report_date,<1y
230,12311,"HUNTINGTON NATIONAL BANK, THE",03/31/2022,19296859.0
537,30810,DISCOVER BANK,03/31/2022,185091.0
635,35301,STATE STREET BANK AND TRUST COMPANY,03/31/2022,20053000.0
1095,60143,COMERICA BANK,03/31/2022,16972000.0
1170,63069,CITY NATIONAL BANK,03/31/2022,10554867.0
...,...,...,...,...
22697,3382547,DEPARTMENT STORES NATIONAL BANK,03/31/2022,39452.0
22787,3402913,BANESCO USA,03/31/2022,233230.0
23496,3783313,CROSS RIVER BANK,03/31/2022,1679060.0
23615,4114567,FIRST REPUBLIC BANK,03/31/2022,18382306.0


In [24]:
df_other_loan_lease_domestic = rcon_data_1[['RSSD9001','RSSD9017', 'RSSD9999', 'RCONA247']]
df_other_loan_lease_domestic = df_other_loan_lease_domestic.rename(columns={
    'RSSD9001': 'Bank_ID',
    'RSSD9017': 'bank_name',
    'RSSD9999': 'report_date',
    'RCONA247': '<1y',
})
df_other_loan_lease_domestic = df_other_loan_lease_domestic.dropna()
df_other_loan_lease_domestic = df_other_loan_lease_domestic[df_other_loan_lease_domestic['report_date'] == '03/31/2022']
df_other_loan_lease_domestic

,Bank_ID,bank_name,report_date,<1y
0,37,BANK OF HANCOCK COUNTY,03/31/2022,5338.0
5,242,FIRST COMMUNITY BANK XENIA-FLORA,03/31/2022,10042.0
10,279,"MINEOLA COMMUNITY BANK, SSB",03/31/2022,19187.0
15,354,BISON STATE BANK,03/31/2022,1849.0
20,457,LOWRY STATE BANK,03/31/2022,11185.0
...,...,...,...,...
23870,5620869,GENESIS BANK,03/31/2022,800.0
23875,5650923,WATERFALL BANK,03/31/2022,3146.0
23880,5660146,ENTREBANK,03/31/2022,0.0
23885,5673191,LOCALITY BANK,03/31/2022,0.0


In [25]:
df_total_other_loan_lease = pd.concat([df_other_loan_lease, df_other_loan_lease_domestic])
df_total_other_loan_lease

,Bank_ID,bank_name,report_date,<1y
230,12311,"HUNTINGTON NATIONAL BANK, THE",03/31/2022,19296859.0
537,30810,DISCOVER BANK,03/31/2022,185091.0
635,35301,STATE STREET BANK AND TRUST COMPANY,03/31/2022,20053000.0
1095,60143,COMERICA BANK,03/31/2022,16972000.0
1170,63069,CITY NATIONAL BANK,03/31/2022,10554867.0
...,...,...,...,...
23870,5620869,GENESIS BANK,03/31/2022,800.0
23875,5650923,WATERFALL BANK,03/31/2022,3146.0
23880,5660146,ENTREBANK,03/31/2022,0.0
23885,5673191,LOCALITY BANK,03/31/2022,0.0


# Asset Decomposition

In [26]:
def sum_assets_and_compute_percentage(df_RMBS_Final, df_loans_first_lien_domestic, df_treasury_and_others, df_other_loan):
    # Sum across rows for each DataFrame
    total_assets_sum = 0
    for df in [df_RMBS_Final, df_loans_first_lien_domestic, df_treasury_and_others, df_other_loan]:
        total = df[['<3m', '3m-1y', '1y-3y', '3y-5y', '5y-15y', '>15y']].sum().sum()
        total_assets_sum += total

    return total_assets_sum

In [27]:
sum_asset =  sum_assets_and_compute_percentage(df_RMBS_Final, df_loans_first_lien_domestic, df_treasury_and_others, df_other_loan)
sum_asset

16590518404.0

In [28]:
sum_asset/total_asset

0.6916024845807683

In [29]:
total_asset

23988517644.0

In [30]:
RMBS = df_RMBS_Final[['<3m', '3m-1y', '1y-3y', '3y-5y', '5y-15y', '>15y']].sum().sum()
a = RMBS / total_asset
a

0.09806049785608004

In [31]:
df_loans = df_loans_first_lien_domestic[['<3m', '3m-1y', '1y-3y', '3y-5y', '5y-15y', '>15y']].sum().sum()
b = df_loans / total_asset
b

0.09277634450066397

In [32]:
df_treasury = df_treasury_and_others[['<3m', '3m-1y', '1y-3y', '3y-5y', '5y-15y', '>15y']].sum().sum()
c = df_treasury / total_asset
c

0.12263495617603573

In [33]:
df_other_ln = df_other_loan[['<3m', '3m-1y', '1y-3y', '3y-5y', '5y-15y', '>15y']].sum().sum()
d = df_other_ln / total_asset
d

0.37813068604798866

In [34]:
df_other_MBS = df_other_MBS_total[['<3y', '>3y']].sum().sum()
e = df_other_MBS  / total_asset
e

0.039776793554338725

In [35]:
df_total_other_ll = df_total_other_loan_lease['<1y'].sum()
f = df_total_other_ll / total_asset
f

0.09353254095553483

In [36]:
a+b+c+d+e #matches with the function above

0.7313792781351072

## roughly the same as the security component (see Panel A)

In [37]:
a+e+c

0.2604722475864545

## roughly the same as the total loan component (see Panel A)

In [38]:
d+b

0.4709070305486526

# Table Analysis (Below are the contents for first column of table 1)

# 1. Aggregate Loss and Bank Loss (include each share)

###  Treasury prices

In [39]:
combined_index_df = pd.read_excel('../data/manual/combined_index_df.xlsx')
treasury_prices = combined_index_df[['date', 'iShares 0-1', 'iShares 1-3', 'sp 3-5', 'iShares 7-10', 'iShares 10-20', 'iShares 20+']]
treasury_prices = treasury_prices.set_index('date')
treasury_prices = treasury_prices.resample('Q').first()
treasury_prices = treasury_prices.loc['2022-03-31':'2023-03-31']
treasury_prices

,iShares 0-1,iShares 1-3,sp 3-5,iShares 7-10,iShares 10-20,iShares 20+
date,,,,,,
2022-03-31,102.099998,113.699997,340.20,108.327118,136.194000,135.810303
2022-06-30,101.980003,110.839996,331.21,101.708519,126.305954,125.121727
2022-09-30,101.980003,110.660004,331.17,98.849396,114.192223,110.068626
2022-12-31,102.279999,108.730003,325.82,93.326981,104.728577,99.298790
2023-03-31,103.129997,109.540001,327.74,93.499557,105.387222,97.738518


In [41]:
df_SP_Treasury_bond_index = pd.read_excel(r'../data/manual/PerformanceGraphExport.xlsx')
#df_SP_Treasury_bond_index['date'] = df_SP_Treasury_bond_index['date'].dt.strftime('%Y-%m-%d')
df_SP_Treasury_bond_index = df_SP_Treasury_bond_index.set_index('date')
df_SP_Treasury_bond_index = df_SP_Treasury_bond_index.resample('Q').first()
df_SP_Treasury_bond_index = df_SP_Treasury_bond_index.loc['2022-03-31':'2023-03-31']
df_SP_Treasury_bond_index

,S&P U.S. Treasury Bond Index
date,
2022-03-31,470.35
2022-06-30,449.59
2022-09-30,438.25
2022-12-31,421.94
2023-03-31,423.71


In [42]:
df_iShare_MBS_ETF = pd.read_csv(r'../data/manual/MBB.csv')
df_iShare_MBS_ETF = df_iShare_MBS_ETF[['Date', 'Adj Close']]
df_iShare_MBS_ETF['Date'] = pd.to_datetime(df_iShare_MBS_ETF['Date'])
df_iShare_MBS_ETF.set_index('Date', inplace=True)
df_iShare_MBS_ETF = df_iShare_MBS_ETF.resample('Q').first()
df_iShare_MBS_ETF.index.rename('date', inplace=True)
df_iShare_MBS_ETF = df_iShare_MBS_ETF.loc['2022-03-31':'2023-03-31']
df_iShare_MBS_ETF

,Adj Close
date,
2022-03-31,100.778877
2022-06-30,95.775497
2022-09-30,93.136818
2022-12-31,88.165436
2023-03-31,89.729256


In [43]:
def RMBs_Multiplier(df_SP_Treasury_bond_index, df_iShare_MBS_ETF):
  
    upper_treasury = df_SP_Treasury_bond_index.loc['2023-03-31', 'S&P U.S. Treasury Bond Index']
    lower_treasury = df_SP_Treasury_bond_index.loc['2022-03-31', 'S&P U.S. Treasury Bond Index']
    
    upper_MBS = df_iShare_MBS_ETF.loc['2023-03-31', 'Adj Close']
    lower_MBS = df_iShare_MBS_ETF.loc['2022-03-31', 'Adj Close']
    
    MBS_change = (upper_MBS / lower_MBS) - 1
    treasury_change = (upper_treasury / lower_treasury) - 1
    multiplier = MBS_change / treasury_change
    
    return multiplier

In [44]:
RMBS_Multiplier = RMBs_Multiplier(df_SP_Treasury_bond_index, df_iShare_MBS_ETF)
RMBS_Multiplier

1.1057080566417443

In [45]:
#Create copies for each dataframe
df_RMBS_Final_1 = df_RMBS_Final.copy() 
df_loans_first_lien_domestic_1 = df_loans_first_lien_domestic.copy()
df_treasury_and_others_1 = df_treasury_and_others.copy()
df_other_loan_1 = df_other_loan.copy()

In [46]:
def report_losses(df_RMBS_Final, df_loans_first_lien_domestic, df_treasury_and_others, df_other_loan, treasury_prices, RMBS_multiplier, gross_asset):
    
    #for df in [df_RMBS_Final, df_loans_first_lien_domestic, df_treasury_and_others, df_other_loan]:
       # df['<1y'] = df['<3m'] + df['3m-1y']
        
    
    price_change = {
        '<1y': treasury_prices.loc['2023-03-31', 'iShares 0-1'] / treasury_prices.loc['2022-03-31', 'iShares 0-1'] - 1,
        '1y-3y': treasury_prices.loc['2023-03-31', 'iShares 1-3'] / treasury_prices.loc['2022-03-31', 'iShares 1-3'] - 1,
        '3y-5y': treasury_prices.loc['2023-03-31', 'sp 3-5'] / treasury_prices.loc['2022-03-31', 'sp 3-5'] - 1,
        '7y-10y': 0.5 * (treasury_prices.loc['2023-03-31', 'iShares 7-10'] / treasury_prices.loc['2022-03-31', 'iShares 7-10'] - 1) + 0.5 * (treasury_prices.loc['2023-03-31', 'iShares 10-20'] / treasury_prices.loc['2022-03-31', 'iShares 10-20'] - 1),
        '>20y': treasury_prices.loc['2023-03-31', 'iShares 20+'] / treasury_prices.loc['2022-03-31', 'iShares 20+'] - 1,
    }

    bucket_mapping = {
        '<3m': '<1y',
        '3m-1y': '<1y',
        '1y-3y': '1y-3y',
        '3y-5y': '3y-5y',
        '5y-15y': '7y-10y',  # Assuming '5y-15y' should be mapped to '7y-10y' based on provided price_change calculation
        '>15y': '>20y',
    }
    
    aggregated_assets = {}
    for name, df in zip(['RMBS', 'Loans', 'Treasury', 'OtherLoan'], 
                        [df_RMBS_Final, df_loans_first_lien_domestic, df_treasury_and_others, df_other_loan]):
        # Ensure columns for aggregation are present
        columns_to_aggregate = [col for col in list(bucket_mapping.keys()) if col in df.columns]
        aggregated_assets[name] = df.groupby(['bank_name', 'Bank_ID'])[columns_to_aggregate].sum().reset_index()
    
    # Initialize DataFrame to store results
    bank_losses_assets = pd.DataFrame(columns=[
        'bank_name', 'bank_ID', 'RMBs_loss', 'treasury_loss', 'loans_loss', 'other_loan_loss', 
        'total_loss', 'Share RMBs', 'Share Treasury and Other', 
        'Share Residential Mortgage', 'Share Other Loan', 'RMBs_asset', 'treasury_asset', 
        'residential_mortgage_asset', 'other_loan_asset', 'core_asset', 'gross_asset', 'loss/core_asset', 'loss/gross_asset',
    ])
    
    for _, gross_row in gross_asset.iterrows():
        bank = gross_row['bank_name']
        bank_id = gross_row['Bank_ID']
        bank_gross_asset = gross_row['gross_asset']
        
        # Initialize variables for loss and asset calculations
        rmbs_loss = loans_loss = treasury_loss = other_loan_loss = total_loss = 0
        rmbs_asset = treasury_asset = loan_asset = other_loan_asset = core_asset = 0
        
        # Process RMBS assets
        if 'RMBS' in aggregated_assets and not aggregated_assets['RMBS'].empty:
            rmbs_row = aggregated_assets['RMBS'][(aggregated_assets['RMBS']['bank_name'] == bank) & (aggregated_assets['RMBS']['Bank_ID'] == bank_id)]
            for bucket, treasury_bucket in bucket_mapping.items():
                if bucket in rmbs_row.columns:
                    asset_amount = rmbs_row.iloc[0][bucket] if not rmbs_row.empty else 0
                    rmbs_loss += (asset_amount * RMBS_multiplier * price_change[treasury_bucket])
                    rmbs_asset += asset_amount
                    
        # Loans
        loans_row = aggregated_assets['Loans'][(aggregated_assets['Loans']['bank_name'] == bank) & (aggregated_assets['Loans']['Bank_ID'] == bank_id)]
        if not loans_row.empty:
            for bucket, treasury_bucket in bucket_mapping.items():
                if bucket in loans_row.columns:
                    asset_amount = loans_row.iloc[0][bucket]
                    loans_loss += (asset_amount * RMBS_multiplier * price_change[treasury_bucket])
                    loan_asset += asset_amount

        # Treasury
        treasury_row = aggregated_assets['Treasury'][(aggregated_assets['Treasury']['bank_name'] == bank) & (aggregated_assets['Treasury']['Bank_ID'] == bank_id)]
        if not treasury_row.empty:
            for bucket, treasury_bucket in bucket_mapping.items():
                if bucket in treasury_row.columns:
                    asset_amount = treasury_row.iloc[0][bucket]
                    treasury_loss += (asset_amount * price_change[treasury_bucket])
                    treasury_asset += asset_amount

        # OtherLoan
        other_loan_row = aggregated_assets['OtherLoan'][(aggregated_assets['OtherLoan']['bank_name'] == bank) & (aggregated_assets['OtherLoan']['Bank_ID'] == bank_id)]
        if not other_loan_row.empty:
            for bucket, treasury_bucket in bucket_mapping.items():
                if bucket in other_loan_row.columns:
                    asset_amount = other_loan_row.iloc[0][bucket]
                    other_loan_loss += (asset_amount * price_change[treasury_bucket])
                    other_loan_asset += asset_amount
                
        total_loss = rmbs_loss + treasury_loss + loans_loss + other_loan_loss
        core_asset = rmbs_asset + treasury_asset + loan_asset + other_loan_asset

        bank_losses_assets.loc[len(bank_losses_assets)] = {
            'bank_name': bank,
            'bank_ID': bank_id,
            'RMBs_loss': rmbs_loss,
            'treasury_loss': treasury_loss,
            'loans_loss': loans_loss,
            'other_loan_loss': other_loan_loss,
            'total_loss': total_loss,
            'Share RMBs': 100 * rmbs_loss / total_loss if total_loss else 0,
            'Share Treasury and Other': 100 * treasury_loss / total_loss if total_loss else 0,
            'Share Residential Mortgage': 100 * loans_loss / total_loss if total_loss else 0,
            'Share Other Loan': 100 * other_loan_loss / total_loss if total_loss else 0,
            'RMBs_asset': rmbs_asset,
            'treasury_asset': treasury_asset,
            'residential_mortgage_asset': loan_asset,
            'other_loan_asset': other_loan_asset,
            'core_asset': core_asset,
            'gross_asset': bank_gross_asset,
            'loss/core_asset': -(total_loss / core_asset) if core_asset else 0,
            'loss/gross_asset': -(total_loss / bank_gross_asset) if bank_gross_asset else 0,
        }

    return bank_losses_assets

In [47]:
bank_losses = report_losses(df_RMBS_Final, df_loans_first_lien_domestic, df_treasury_and_others, df_other_loan, treasury_prices, RMBS_Multiplier, gross_asset)
bank_losses

,bank_name,bank_ID,RMBs_loss,treasury_loss,loans_loss,other_loan_loss,total_loss,Share RMBs,Share Treasury and Other,Share Residential Mortgage,Share Other Loan,RMBs_asset,treasury_asset,residential_mortgage_asset,other_loan_asset,core_asset,gross_asset,loss/core_asset,loss/gross_asset
0,"HUNTINGTON NATIONAL BANK, THE",12311,-7.275189e+06,-7.555621e+05,-5.025313e+06,-3.448913e+06,-1.650498e+07,44.078759,4.577783,30.447259,20.896200,23910558.0,6623679.0,22529389,92122592.0,145186218.0,176020595.0,0.113681,0.093767
1,DISCOVER BANK,30810,-8.381464e+04,-1.304031e+05,-2.292996e+05,-1.609604e+06,-2.053122e+06,4.082303,6.351455,11.168338,78.397904,364810.0,5612015.0,804718,92452467.0,99234010.0,105657408.0,0.020690,0.019432
2,STATE STREET BANK AND TRUST COMPANY,35301,-6.593558e+06,-2.229374e+06,0.000000e+00,-3.003046e+05,-9.123237e+06,72.272141,24.436213,-0.000000,3.291646,22436000.0,62518000.0,0,35293000.0,120247000.0,318494000.0,0.075871,0.028645
3,COMERICA BANK,60143,-2.823337e+06,-4.300227e+05,-3.050844e+05,5.455126e+04,-3.503893e+06,80.577151,12.272712,8.707013,-1.556876,9378000.0,4835000.0,1825000,47475000.0,63513000.0,89142000.0,0.055168,0.039307
4,CITY NATIONAL BANK,63069,-7.167800e+04,-3.352253e+05,-4.150180e+06,-1.322681e+06,-5.879765e+06,1.219062,5.701339,70.584119,22.495480,336792.0,5698026.0,19072816,39175374.0,64283008.0,91770285.0,0.091467,0.064070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4839,GENESIS BANK,5620869,0.000000e+00,0.000000e+00,0.000000e+00,-6.670438e+03,-6.670438e+03,-0.000000,-0.000000,-0.000000,100.000000,0.0,0.0,0,59836.0,59836.0,95052.0,0.111479,0.070177
4840,WATERFALL BANK,5650923,-2.054440e+03,-4.925118e+02,0.000000e+00,-2.847616e+03,-5.394568e+03,38.083498,9.129773,-0.000000,52.786729,6628.0,2713.0,0,24447.0,33788.0,87152.0,0.159659,0.061898
4841,ENTREBANK,5660146,0.000000e+00,0.000000e+00,0.000000e+00,-1.140152e+02,-1.140152e+02,-0.000000,-0.000000,-0.000000,100.000000,0.0,0.0,0,3113.0,3113.0,35261.0,0.036626,0.003233
4842,LOCALITY BANK,5673191,0.000000e+00,0.000000e+00,0.000000e+00,-5.035088e+02,-5.035088e+02,-0.000000,-0.000000,-0.000000,100.000000,0.0,0.0,0,8312.0,8312.0,43775.0,0.060576,0.011502


In [48]:
median_percentage = bank_losses[['Share RMBs', 'Share Treasury and Other', 
        'Share Residential Mortgage', 'Share Other Loan']].median()
median_percentage
#close to reported values, but the distribution looks good

Share RMBs                     8.038755
Share Treasury and Other      18.238954
Share Residential Mortgage    17.832035
Share Other Loan              35.115753
dtype: float64

In [49]:
std_percentages =bank_losses[['Share RMBs', 'Share Treasury and Other', 
        'Share Residential Mortgage', 'Share Other Loan']].std()
std_percentages #this is slightly different 

Share RMBs                    17.493873
Share Treasury and Other      43.213003
Share Residential Mortgage    22.415385
Share Other Loan              42.339553
dtype: float64

## Total, median, and std of losses

In [50]:
total_sum_loss = bank_losses['total_loss'].sum()
total_sum_loss

-1756241754.401683

In [51]:
median_bank_loss = bank_losses['total_loss'].median()
median_bank_loss 

-24668.349052336016

In [52]:
std_bank_loss = bank_losses['total_loss'].std()
std_bank_loss

5715576.773707272

## Checks

In [53]:
core_asset = bank_losses['core_asset'].sum()
core_asset

16590518404.0

In [54]:
gross_asset = bank_losses['gross_asset'].sum()
gross_asset 

23988517644.0

In [55]:
core_asset / gross_asset

0.6916024845807683

# 2. Loss / Asset

## Ratio Results

In [56]:
median_loss_asset_ratio = bank_losses['loss/gross_asset'].median()
median_loss_asset_ratio

0.07832787629535784

In [57]:
average_loss_asset_ratio = bank_losses['loss/gross_asset'].mean()
average_loss_asset_ratio

0.08346929927154349

In [58]:
std_loss_asset_ratio = bank_losses['loss/gross_asset'].std()
std_loss_asset_ratio

0.04505958314654051

# 3. Uninsured Deposit/MM Asset

In [59]:
uninsured_deposit = rcon_data_1[['RSSD9001','RSSD9017', 'RSSD9999', 'RCON5597']]
uninsured_deposit = uninsured_deposit .rename(columns={
    'RSSD9001': 'bank_ID',
    'RSSD9017': 'bank_name',
    'RSSD9999': 'report_date',
    'RCON5597': 'uninsured_deposit'

})
uninsured_deposit = uninsured_deposit[uninsured_deposit['report_date'] == '03/31/2022']
uninsured_deposit['uninsured_deposit'].sum()/gross_asset #Similar to Panel A result of 37.4%

0.3579107245147957

In [60]:
uninsured_deposit['uninsured_deposit'].sum() #Roughly 9 trillion 

8585747730.0

In [61]:
def calculate_uninsured_deposit_mm_asset(uninsured_deposit, bank_losses):
    
    # Initialize an empty list to store the results
    results = []
    
    # Adjust the uninsured_deposit DataFrame to use both 'bank_name' and 'Bank_ID' as a multi-index for quick lookup
    uninsured_lookup = uninsured_deposit.set_index(['bank_name', 'bank_ID'])['uninsured_deposit'].to_dict()
    
    # Iterate over each row in bank_losses DataFrame
    for index, bank_loss_row in bank_losses.iterrows():
        bank_name = bank_loss_row['bank_name']
        bank_id = bank_loss_row['bank_ID']
        
        # Adjust the lookup to include 'Bank_ID'
        uninsured_deposit_value = uninsured_lookup.get((bank_name, bank_id), 0)
        
        # Calculate 'MM Asset' as the sum of 'total_loss' and 'gross_asset' (as defined in the paper)
        mm_asset = bank_loss_row['total_loss'] + bank_loss_row['gross_asset']
        
        # Calculate Uninsured Deposit/MM Asset ratio 
        if mm_asset > 0:
            uninsured_deposit_mm_asset_ratio = uninsured_deposit_value / mm_asset
        
        # Append to final dataframe
        results.append({
            'bank_name': bank_name,
            'bank_ID': bank_id, 
            'total_loss': bank_loss_row['total_loss'], 
            'total_asset': bank_loss_row['gross_asset'],
            'mm_asset': mm_asset,
            'uninsured_deposit': uninsured_deposit_value, 
            'Uninsured_Deposit_MM_Asset': uninsured_deposit_mm_asset_ratio
        })
    
    # Convert results list to DataFrame and sort by 'Bank_ID'
    results_df = pd.DataFrame(results).sort_values(by=['bank_name', 'bank_ID'])
    
    return results_df

un_mm_ratio = calculate_uninsured_deposit_mm_asset(uninsured_deposit, bank_losses)

## Checks

In [62]:
un_mm_ratio['total_loss'].sum()

-1756241754.4016833

In [63]:
un_mm_ratio['total_asset'].sum()

23988517644.0

In [64]:
un_mm_ratio['mm_asset'].sum()

22232275889.598316

## Ratio Results

In [65]:
un_mm_ratio['Uninsured_Deposit_MM_Asset'].median()

0.3188222083530178

In [66]:
un_mm_ratio['Uninsured_Deposit_MM_Asset'].std()

0.16733349291357616

# 4. Insured Deposit Coverage Ratio - CURRENTLY NOT WORKING

In [67]:
insured_deposit = rcon_data_1[['RSSD9001','RSSD9017', 'RSSD9999', 'RCONF049', 'RCONF045']] 
#RCFDF049 are Deposit accounts (excluding retirement accounts) of $250,000 or less
#RCFDF045 are Retirement deposit accounts of $250,000 or less

insured_deposit = insured_deposit.rename(columns={
    'RSSD9001': 'bank_ID',
    'RSSD9017': 'bank_name',
    'RSSD9999': 'report_date',
    'RCONF049': 'insured_deposit_1',
    'RCONF045': 'insured_deposit_2'
})
insured_deposit = insured_deposit[insured_deposit['report_date'] == '03/31/2022']
insured_deposit['insured_deposit_1'].sum()/total_asset + insured_deposit['insured_deposit_2'].sum()/total_asset

#THESE NUMBERS ARE SLIGHLY FAR FROM INSURED DEPOSIT IN PANEL A

0.33296701528357264

In [68]:
insured_deposit['insured_deposit'] = insured_deposit['insured_deposit_1'] + insured_deposit['insured_deposit_2']
insured_deposit 

,bank_ID,bank_name,report_date,insured_deposit_1,insured_deposit_2,insured_deposit
0,37,BANK OF HANCOCK COUNTY,03/31/2022,57430.0,2428.0,59858.0
5,242,FIRST COMMUNITY BANK XENIA-FLORA,03/31/2022,33454.0,1385.0,34839.0
10,279,"MINEOLA COMMUNITY BANK, SSB",03/31/2022,197561.0,11879.0,209440.0
15,354,BISON STATE BANK,03/31/2022,9794.0,0.0,9794.0
20,457,LOWRY STATE BANK,03/31/2022,34905.0,2229.0,37134.0
...,...,...,...,...,...,...
23870,5620869,GENESIS BANK,03/31/2022,9402.0,0.0,9402.0
23875,5650923,WATERFALL BANK,03/31/2022,7911.0,0.0,7911.0
23880,5660146,ENTREBANK,03/31/2022,1020.0,0.0,1020.0
23885,5673191,LOCALITY BANK,03/31/2022,912.0,0.0,912.0


In [69]:
insured_deposit['insured_deposit'].sum()

7987385121.0

In [70]:
def insured_deposit_coverage_ratio(insured_deposit, uninsured_deposit, bank_losses):
    # Initialize an empty list to store the results
    results = []
    
    # Create dictionaries from insured and uninsured deposits for quick lookup
    insured_lookup = insured_deposit.set_index(['bank_name', 'bank_ID'])['insured_deposit'].to_dict()
    uninsured_lookup = uninsured_deposit.set_index(['bank_name', 'bank_ID'])['uninsured_deposit'].to_dict()
    
    # Iterate over each row in bank_losses DataFrame
    for index, bank_loss_row in bank_losses.iterrows():
        bank_name = bank_loss_row['bank_name']
        bank_id = bank_loss_row['bank_ID']
        
        # Retrieve insured and uninsured deposit values
        insured_deposit_value = insured_lookup.get((bank_name, bank_id), 0)
        uninsured_deposit_value = uninsured_lookup.get((bank_name, bank_id), 0)
        
        # Calculate mark-to-market asset value as the sum of 'total_asset' minus 'total_loss'
        mark_to_market_asset_value = bank_loss_row['gross_asset'] + bank_loss_row['total_loss']
        
        # Calculate the insured deposit coverage ratio
        if insured_deposit_value > 0:  # Prevent division by zero
            coverage_ratio = (mark_to_market_asset_value - uninsured_deposit_value - insured_deposit_value) / insured_deposit_value
        
        # Append the result
        results.append({
            'bank_name': bank_name,
            'bank_ID': bank_id,
            'mm_asset': mark_to_market_asset_value,
            'insured_deposit': insured_deposit_value,
            'uninsured_deposit': uninsured_deposit_value,
            'insured_deposit_coverage_ratio': coverage_ratio
        })
    
    # Convert results list to DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df

coverage_ratios_df = insured_deposit_coverage_ratio(insured_deposit, uninsured_deposit, bank_losses)
coverage_ratios_df

,bank_name,bank_ID,mm_asset,insured_deposit,uninsured_deposit,insured_deposit_coverage_ratio
0,"HUNTINGTON NATIONAL BANK, THE",12311,1.595156e+08,78986097.0,83160026.0,-0.033303
1,DISCOVER BANK,30810,1.036043e+08,57543068.0,8691421.0,0.649423
2,STATE STREET BANK AND TRUST COMPANY,35301,3.093708e+08,11687000.0,155822000.0,12.138424
3,COMERICA BANK,60143,8.563811e+07,21921000.0,51044000.0,0.578126
4,CITY NATIONAL BANK,63069,8.589052e+07,17928917.0,58096403.0,0.550240
...,...,...,...,...,...,...
4839,GENESIS BANK,5620869,8.838156e+04,9402.0,NaN,NaN
4840,WATERFALL BANK,5650923,8.175743e+04,7911.0,NaN,NaN
4841,ENTREBANK,5660146,3.514698e+04,1020.0,NaN,NaN
4842,LOCALITY BANK,5673191,4.327149e+04,912.0,NaN,NaN


## Checks

In [71]:
coverage_ratios_df['insured_deposit'].sum()

7987385121.0

In [72]:
coverage_ratios_df['uninsured_deposit'].sum()

8585747730.0

In [73]:
coverage_ratios_df['mm_asset'].sum()

22232275889.59832

## Ratio Results

In [74]:
coverage_ratios_df['insured_deposit_coverage_ratio'].median()

0.4268269722661211

In [75]:
coverage_ratios_df['insured_deposit_coverage_ratio'].std()

332.1760833543981